In [25]:
import random as rn
import numpy as np
from numpy.random import choice as np_choice

class AntColony(object):

    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        """
        Args:
            distances (2D numpy.array): Matrice quadrata delle distanze. Si assume np.inf nella diagonale.
            n_ants (int): Numero di agenti per ogni iterazione
            n_best (int): Numero di agenti con migliori performance che possono rilasciare feromone
            n_iteration (int): Numero iterazioni
            decay (float): Pheromone decay (es. 0.95). Il valore del feromone viene moltiplicato per il decay.
            alpha (int or float): valore dell'esponente per il feronome (default = 1)
            beta (int or float): valore dell'esponente per la distanza (default = 1)

        Esempio:
            ant_colony = AntColony(german_distances, 100, 20, 2000, 0.95, alpha=1, beta=2)          
        """
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    # funzione principale richiamata dall'esterno della classe una volta
    # inizializzata la classe
    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            #print (shortest_path)
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path            
            self.pheromone = self.pheromone * self.decay            
        return all_time_shortest_path

    # aggiornamento del feromone
    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    # calcolo della distanza
    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start)) # going back to where we started    
        return path

    # determina la mossa per ogni agente
    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move

In [ ]:
"""
distances = np.array([[np.inf, 2, 2, 5, 7, 3, 5, 4, 7, 1, 5],
                      [2, np.inf, 4, 8, 2, 2, 6, 4, 6, 2, 3],
                      [2, 4, np.inf, 1, 3, 4, 2, 6, 3, 4, 1],
                      [5, 8, 1, np.inf, 2, 3, 4, 2, 5, 4, 5],
                      [7, 3, 4, 2, np.inf, 4, 3, 3, 2, 3, 1],
                      [3, 1, 5, 5, 3, np.inf, 3, 4, 4, 6, 6],
                      [4, 3, 6, 4, 3, 4, np.inf, 5, 4, 3, 5],
                      [5, 2, 3, 4, 2, 4, 2, np.inf, 2, 1, 7],
                      [2, 4, 4, 3, 5, 5, 3, 5, np.inf, 2, 1],
                      [7, 2, 2, 3, 2, 3, 5, 2, 4, np.inf, 4],
                      [4, 3, 1, 1, 5, 2, 4, 5, 5, 3, np.inf]])
"""

In [69]:
lst = []
size = 50
for i in range(size):
    row = []
    for j in range(size):
        if i == j:
            row.append(np.inf)
        else:
            row.append(np.random.randint(1, 8))
    lst.append(row)
distances = np.array(lst)
distances

array([[inf,  7.,  6., ...,  2.,  3.,  3.],
       [ 6., inf,  2., ...,  2.,  6.,  4.],
       [ 6.,  1., inf, ...,  3.,  1.,  7.],
       ...,
       [ 2.,  4.,  5., ..., inf,  7.,  6.],
       [ 7.,  2.,  1., ...,  7., inf,  7.],
       [ 1.,  7.,  4., ...,  1.,  5., inf]])

In [71]:
ant_colony = AntColony(distances, 30, 10, 300, 0.95, alpha=1, beta=1)
shortest_path = ant_colony.run()
print ("shorted_path: {}".format(shortest_path))
costs = []
for edge in shortest_path[0]:
    costs.append(distances[edge[0]][edge[1]])
print('costs:', costs)

shorted_path: ([(0, 36), (36, 37), (37, 19), (19, 20), (20, 16), (16, 24), (24, 5), (5, 44), (44, 3), (3, 39), (39, 34), (34, 14), (14, 49), (49, 23), (23, 4), (4, 21), (21, 46), (46, 2), (2, 35), (35, 25), (25, 42), (42, 11), (11, 29), (29, 45), (45, 15), (15, 10), (10, 33), (33, 13), (13, 6), (6, 26), (26, 32), (32, 7), (7, 40), (40, 27), (27, 48), (48, 17), (17, 12), (12, 31), (31, 18), (18, 28), (28, 22), (22, 8), (8, 9), (9, 41), (41, 30), (30, 1), (1, 38), (38, 47), (47, 43), (43, 0)], 50.0)
costs: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
